In [3]:
import argparse
from artifacts.predict_tag import predict_tag, load_data, tfidf_transform
import json

In [10]:
import numpy as np
import pandas as pd
import csv

In [6]:
data_col = 'intent'

df = pd.read_json('sample_data/conala-test.json')
df = df[[data_col]].rename(columns={data_col: 'code_block'})

In [15]:
df.to_csv('sample_data/input.csv', index=False, quoting=csv.QUOTE_ALL)

In [16]:
path_to_model_dir = '.'
data_input_path = 'sample_data/'
data_output_path = 'sample_data/'
model_type = 'svm'

In [17]:
predict_tag(data_input_path, data_output_path, path_to_model_dir, model_type)

loaded
the model has been loaded
predictions were calculated
finished


In [19]:
out_df = pd.read_csv('sample_data/output.csv')
out_df[out_df['visualization']==1].head(10)

# Lime

In [41]:
import cloudpickle
import pickle
import os
from lime import lime_text
from sklearn.pipeline import make_pipeline
from lime.lime_text import LimeTextExplainer

In [39]:
DATASET_PATH = os.path.join(data_input_path, "input.csv")
MODEL = "svm"
GRAPH_VER = 5
# svm or logreg

TAGS_TO_PREDICT = [
    "import",
    "data_import",
    "data_export",
    "preprocessing",
    "visualization",
    "model",
    "deep_learning_model",
    "train",
    "predict",
]

MODEL_DIR = os.path.join(
    path_to_model_dir,
    "artifacts/models/{}_regex_graph_v{}.sav".format(MODEL, GRAPH_VER),
)
TFIDF_DIR = os.path.join(
    path_to_model_dir,
    "artifacts/models/tfidf_{}_graph_v{}.pickle".format(MODEL, GRAPH_VER),
)
CODE_COLUMN = "code_block"

df = load_data(DATASET_PATH)
code_blocks = df[CODE_COLUMN]

In [35]:
tfidf = cloudpickle.load(open(TFIDF_DIR, "rb"))
clf = pickle.load(open(MODEL_DIR, "rb"))

/var/folders/63/q32zgnw527g9547scbl9tby00000gn/T/ipykernel_53643/2354969462.py:1: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  tfidf = cloudpickle.load(open(TFIDF_DIR, "rb"))
/var/folders/63/q32zgnw527g9547scbl9tby00000gn/T/ipykernel_53643/2354969462.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  clf = pickle.load(open(MODEL_DIR, "rb"))


In [40]:
c = make_pipeline(tfidf, clf)

In [53]:
c.predict(df[CODE_COLUMN])

AttributeError: 'MultiOutputRegressor' object has no attribute 'predict_log_proba'

In [74]:
c2 = make_pipeline(tfidf, clf.estimators_[3])

In [75]:
explainer = LimeTextExplainer()

In [79]:
idx = 84
exp = explainer.explain_instance(df[CODE_COLUMN][idx], c2.predict_proba, num_features=6)
print('Document id: %d' % idx)
print('Probability(christian) =', c2.predict_proba([df[CODE_COLUMN][idx]])[0][1])

Document id: 84
Probability(christian) = 0.0


In [85]:
exp.save_to_file('HJDHW.html', text=True)

# Predict Tag (full inference)

In [ ]:
parser = argparse.ArgumentParser(
    description="Run pipeline for model NL2ML (tag prediction)"
)
parser.add_argument(
    "-p", "--params", type=str, help="path to request.json file with parameters"
)

args = parser.parse_args()
with open(args.params) as jf:
    params = json.load(jf)


model_types = [
    "svm",
    # "logreg",
]

path_to_model_dir = params["path_to_model_dir"]
data_input_path = params["data_input_path"]
data_output_path = params["data_output_path"]

model_type = params["model_parameters"].get("model_type", "svm")

predict_tag(data_input_path, data_output_path, path_to_model_dir, model_type)